# Infrastructure

### TOC
- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Create the Azure Resource Group](#1)
- [2️⃣ Create deployment using 🦾 Bicep](#2)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [1]:
import os
import json
import datetime
import requests

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "eastus2"
openai_resources = [ {"name": "openai1", "location": "eastus"} ] # list of OpenAI resources to deploy.
openai_resources_sku = "S0"
openai_model_name = "gpt-4o"
openai_model_version = "2024-11-20"
openai_deployment_name = "gpt-4o"
openai_api_version = "2024-02-01"
openai_specification_url='https://raw.githubusercontent.com/Azure/azure-rest-api-specs/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/stable/' + openai_api_version + '/inference.json'

# variables specific for the vector search lab
log_analytics_name = "workspace"
app_insights_name = 'insights'
openai_embeddings_deployment_name = "text-embedding-3-large"
openai_embeddings_model_name = "text-embedding-3-large"
openai_embeddings_model_version = "1"
searchservice_resource_name = "search"
searchservice_sku = "standard"
administratorLogin = "labuser"
administratorLoginPassword = "Password1234!"
storageContainerName = "data"
principalId = ""
if not principalId:
    principalId = !az ad signed-in-user show --query id -o tsv
    principalId = principalId[0]

<a id='1'></a>
### 1️⃣ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [2]:
resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location}
if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print("✅ Azure Resource Group ", resource_group_name, " created ⌚ ", datetime.datetime.now().time())

✅ Azure Resource Group  lab-infra  created ⌚  10:17:02.370198


<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [3]:

if len(openai_resources) > 0:
    backend_id = openai_resources[0].get("name")

bicep_parameters = {
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {
    "openAIConfig": { "value": openai_resources },
    "openAIDeploymentName": { "value": openai_deployment_name },
    "openAISku": { "value": openai_resources_sku },
    "openAIModelName": { "value": openai_model_name },
    "openAIModelVersion": { "value": openai_model_version },
    "logAnalyticsName": { "value": log_analytics_name },
    "openAIEmbeddingsDeploymentName": { "value": openai_embeddings_deployment_name},
    "openAIEmbeddingsModelName": { "value": openai_embeddings_model_name},
    "openAIEmbeddingsModelVersion": { "value": openai_embeddings_model_version},
    "searchServiceName": { "value": searchservice_resource_name},
    "searchServiceSku": { "value": searchservice_sku},
    "administratorLogin": { "value": administratorLogin },
    "administratorLoginPassword": { "value": administratorLoginPassword },
    "storageContainerName": { "value": storageContainerName },
    "principalId": { "value": principalId }  
  }
}
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))
! az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file "main.bicep" --parameters "params.json"

{
  "id": "/subscriptions/06d043e2-5a2e-46bf-bf48-fffee525f377/resourceGroups/lab-infra/providers/Microsoft.Resources/deployments/infra",
  "location": null,
  "name": "infra",


  "properties": {
    "correlationId": "0d57e2e8-03e9-4a44-b9e1-4ba16fb3ea56",
    "debugSetting": null,
    "dependencies": [
      {
        "dependsOn": [
          {
            "id": "/subscriptions/06d043e2-5a2e-46bf-bf48-fffee525f377/resourceGroups/lab-infra/providers/Microsoft.OperationalInsights/workspaces/workspace-lzlxtia67q7hc",
            "resourceGroup": "lab-infra",
            "resourceName": "workspace-lzlxtia67q7hc",
            "resourceType": "Microsoft.OperationalInsights/workspaces"
          }
        ],
        "id": "/subscriptions/06d043e2-5a2e-46bf-bf48-fffee525f377/resourceGroups/lab-infra/providers/Microsoft.Insights/components/insights-lzlxtia67q7hc",
        "resourceGroup": "lab-infra",
        "resourceName": "insights-lzlxtia67q7hc",
        "resourceType": "Microsoft.Insights/components"
      },
      {
        "dependsOn": [
          {
            "id": "/subscriptions/06d043e2-5a2e-46bf-bf48-fffee525f377/resourceGroups/lab-infra/providers/Micros

        "dependsOn": [
          {
            "id": "/subscriptions/06d043e2-5a2e-46bf-bf48-fffee525f377/resourceGroups/lab-infra/providers/Microsoft.CognitiveServices/accounts/openai1-lzlxtia67q7hc",
            "resourceGroup": "lab-infra",
            "resourceName": "openai1-lzlxtia67q7hc",
            "resourceType": "Microsoft.CognitiveServices/accounts"
          }
        ],
        "id": "/subscriptions/06d043e2-5a2e-46bf-bf48-fffee525f377/resourceGroups/lab-infra/providers/Microsoft.CognitiveServices/accounts/openai1-lzlxtia67q7hc/deployments/gpt-4o",
        "resourceGroup": "lab-infra",
        "resourceName": "openai1-lzlxtia67q7hc/gpt-4o",
        "resourceType": "Microsoft.CognitiveServices/accounts/deployments"
      },
      {
        "dependsOn": [
          {
            "id": "/subscriptions/06d043e2-5a2e-46bf-bf48-fffee525f377/resourceGroups/lab-infra/providers/Microsoft.CognitiveServices/accounts/openai1-lzlxtia67q7hc",
            "resourceGroup": "lab-infra",
 